In [1]:
import pandas as pd
import seaborn as sns

import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import pearsonr
from scipy.stats import spearmanr

# Data collection and preprosessing

For this project we will collect data from three sources. All three data sets are of protein sequences with experimental TM (melting temperature). The three data sets will all be combined before further processing.

## Meltome atlas data

This data was provided as a csv file with gene names and uniprot ids and TM values. Thus we need to first download and map the sequences for this data set.

In [2]:
file_c="../data/MeltomAtlas/cross-species.csv"

df_c = pd.read_csv(file_c)


df_c = df_c.dropna(subset=['meltPoint'])
df_c_nonan = df_c.dropna()

# write query column with uniprot id
df_c_nonan['query'] = df_c_nonan['Protein_ID'].apply(lambda x: x.split('_')[0])

# Drop duplicates
df_c_noredun = df_c_nonan.drop_duplicates(subset=['query'])
df_TM = df_c_nonan.groupby('query')['meltPoint'].median().to_frame()

/tmp/ipykernel_27010/2097973913.py:3: DtypeWarning: Columns (1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_c = pd.read_csv(file_c)
/tmp/ipykernel_27010/2097973913.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_c_nonan['query'] = df_c_nonan['Protein_ID'].apply(lambda x: x.split('_')[0])


### Separate human part

Human part needs to be treated diferently since only gene name is used as identifier not uniprot id

In [6]:
df_h = df_c[df_c['run_name'].str.match('Homo sapiens')]

### Make the data non redundant

There are several measurments of the same gene, so we need to only grab the median value of the measured melting temperatures for each gene.

In [7]:
df_h_noredu = df_h.groupby('gene_name')['meltPoint'].median().to_frame()
df_h_noredu.to_csv("human_TM.csv")

### Save text file with gene names

In [7]:
with open("../data/MeltomAtlas/gene_names.txt", "w") as file_writer:
    for row in df_h_noredu.iterrows():
        gene_name = row[0]
        file_writer.write(f"{gene_name}\n")

### Save ID non human

In [11]:
with open("../data/MeltomAtlas/gene_names_nohuman.txt", "w") as file_writer:
    for row in df_c_nonan.iterrows():
        gene_name = row[1]['Protein_ID']
        file_writer.write(f"{gene_name.split('_')[0]}\n")

### Download seq data from ids

In [3]:

# fetch all ids from the nonredundant df
list_ids = []
for row in df_c_noredun.iterrows():
    gene_name = row[1]['Protein_ID']
    list_ids.append(f"{gene_name.split('_')[0]}")

In [5]:
from Bio import SeqIO
from multiprocessing import Pool

# Fetch all ids from uniprot
IDS = [f"{id_}"  for idx, id_ in enumerate(list_ids)]

def fetch(id_):
    rec  = ! curl -s https://rest.uniprot.org/uniprotkb/{id_}.fasta
    return rec
with Pool(16) as p:
    list_rec = p.map(fetch, IDS)
#print(f"Done fetching all {idx} sequences")

In [6]:
df_rec = {}

for rec in list_rec:
    if len(rec) > 2:
        id_ = rec[0].split("|")[1]
        df_rec[id_]=("".join([s for s in rec[1:]]) , rec[0])

len(df_rec)

24262

### Write fasta file

In [11]:

#Write all downloaded entries 

data_set = "meltome_atlas"
with open(f"../data/MeltomAtlas/{data_set}_TM.fasta", "w") as file_writer:
    for rec in df_TM.iterrows():
        id_ = rec[0]
        TM = rec[1]['meltPoint']


        try:
            seq = df_rec[id_][0]
            #print(f"ID: {id_}\tTM: {TM}\tSeq: {seq}")
            file_writer.write(f">{id_} {data_set} {TM}\n{seq}\n")
            
        except KeyError:
            #print(f"Id: {id_} does not exist")
            print(f"Skiping entry {rec}")
        

Id: A0A0A6YY22 does not exist
Id: A0A0B4J1K8 does not exist
Id: A0A0G2JEB8 does not exist
Id: A0A0H2UKQ1 does not exist
Id: A0A0K2H409 does not exist
Id: A0A0K2H416 does not exist
Id: A0A0K2H440 does not exist
Id: A0A0K2H450 does not exist
Id: A0A0K2H482 does not exist
Id: A0A0K2H4B3 does not exist
Id: A0A0K2H4B6 does not exist
Id: A0A0K2H4C3 does not exist
Id: A0A0K2H4F8 does not exist
Id: A0A0K2H4H8 does not exist
Id: A0A0K2H4L3 does not exist
Id: A0A0K2H4P5 does not exist
Id: A0A0K2H4P8 does not exist
Id: A0A0K2H4Q8 does not exist
Id: A0A0K2H4T3 does not exist
Id: A0A0K2H4T8 does not exist
Id: A0A0K2H4U2 does not exist
Id: A0A0K2H4U6 does not exist
Id: A0A0K2H4V9 does not exist
Id: A0A0K2H4W0 does not exist
Id: A0A0K2H4W1 does not exist
Id: A0A0K2H4W4 does not exist
Id: A0A0K2H4W7 does not exist
Id: A0A0K2H4W9 does not exist
Id: A0A0K2H4X0 does not exist
Id: A0A0K2H4X4 does not exist
Id: A0A0K2H4Y0 does not exist
Id: A0A0K2H4Y3 does not exist
Id: A0A0K2H4Y7 does not exist
Id: A0A0K2

In [12]:
# Check the total written entries in fasta file
! grep "^>" TM.fasta | wc

  24262   72786  950233


## Combine Meltome atlas data with data from the other sets

In [13]:
! if [ -f "TM.fasta" ]; then mv TM.fasta ../data/data_Tm/meltome_atlas_TM.fasta; fi


In [12]:
! cat ../data/MeltomAtlas/meltome_atlas_TM.fasta > ../data/all_tms.fasta
! cat ../data/JarZab/cleaned_enzyme_tms_jarzab_v1.fasta >> ../data/all_tms.fasta
! cat ../data/Leuenberger/cleaned_enzyme_tms_v1.fasta >> ../data/all_tms.fasta

## Make the combined data set non redundant
When Using CD-Hit (95%)

In [32]:
! cd-hit -i ../data/all_tms.fasta -o ../data/all_tms_noredun.fasta -c 0.99 -T 10

Program: CD-HIT, V4.8.1 (+OpenMP), Apr 07 2021, 10:57:21
Command: cd-hit -i ../data/all_tms.fasta -o
         ../data/all_tms_noredun.fasta -c 0.99 -T 10

Started: Mon Jun 12 18:22:30 2023
                            Output                              
----------------------------------------------------------------
total seq: 68493
longest and shortest : 35213 and 31
Total letters: 33259678
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 41M
Buffer          : 10 X 18M = 188M
Table           : 2 X 66M = 132M
Miscellaneous   : 0M
Total           : 363M

Table limit with the given memory limit:
Max number of representatives: 592200
Max number of word counting entries: 54572295

# comparing sequences from          0  to       4557
....---------- new table with     3058 representatives
# comparing sequences from       4557  to       9885
----------   3846 remaining sequences to the next cycle
---------- new table with      932 representatives
# compa

In [43]:
## Removing sequences longer than 780 residues (since my GPU cant handle them)
## We also filter sequences with Non standard residues
import math
NON_STANDARD_AMINO = ["B", "U", "Z", "X"]

from Bio import SeqIO
df = {"id":[], "TM": [], "seq":[], "Data_set":[]}
for rec in SeqIO.parse("../data/all_tms_noredun.fasta", "fasta"):
    df["id"].append(rec.id)
    df["TM"].append(math.floor(float(rec.description.split()[-1])))
    df["Data_set"].append(rec.description.split()[-2])
    df["seq"].append(str(rec.seq))
df = pd.DataFrame(df)
with open("../data/all_tms_noredun_780.fasta", "w") as f:
    for row in df.iterrows():
        seq = row[1]["seq"]
        if len(seq) > 780:
            print(f"Removing {row[1]['id']} due to length")
            continue
        if any(amino in seq for amino in NON_STANDARD_AMINO):
            print(f"Removing {row[1]['id']} due to non standard res")
            continue
        id_ = row[1]["id"]
        seq = row[1]["seq"]
        TM  = row[1]["TM"]
        da  = row[1]["Data_set"]
        
        f.write(f">{id_} {da} {TM}\n{seq}\n")
        
if any(amino in sequence for amino in NON_STANDARD_AMINO):
    raise ValueError (f"Sequences can not contain non standard amino acids")

Removing A0A061ACR1 due to length
Removing A0A061AD29 due to length
Removing A0A087WP63 due to length
Removing A0A087WPH5 due to non standard res
Removing A0A087WPL5 due to length
Removing A0A087WPR9 due to non standard res
Removing A0A087WPX0 due to length
Removing A0A087WQ86 due to length
Removing A0A087WR24 due to length
Removing A0A087WR75 due to length
Removing A0A087WRE5 due to non standard res
Removing A0A087WRE7 due to non standard res
Removing A0A087WRM0 due to non standard res
Removing A0A087WRV0 due to non standard res
Removing A0A087WS03 due to length
Removing A0A0A0MPG3 due to non standard res
Removing A0A0A0MQ73 due to length
Removing A0A0A0MQ79 due to length
Removing A0A0A0MQ98 due to length
Removing A0A0A0MQA5 due to non standard res
Removing A0A0A0MQD1 due to length
Removing A0A0A0MQE5 due to length
Removing A0A0A6YVS2 due to non standard res
Removing A0A0A6YW28 due to length
Removing A0A0A6YWG7 due to length
Removing A0A0A6YX25 due to non standard res
Removing A0A0A6Y

NameError: name 'sequence' is not defined

In [34]:
! grep "^>" ../data/all_tms.fasta| wc

  68493  205479 2000956


In [35]:
! grep "^>" ../data/all_tms_noredun.fasta| wc

  40193  120579 1347777


In [36]:
! grep "^>" ../data/all_tms_noredun_780.fasta| wc

  33568  100704  966311
